In [1]:

## TODO LIST

# move everything to absolute space
# handle rays not colliding better
# add back regression priors
# resample between profile shapes
# better batches normal / intersection code
# export polygon data for freecad import
# build 3D model
# tests